<a href="https://colab.research.google.com/github/marcusflygar1-hash/AH2179_Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing the neccesary libraries for the project.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error, r2_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

Okey-



Psuedo Kod.
rensa och "clean" datan. Ta bort all null values och NaN's.

Dela sedan upp i 5 minuters tidsperioder.



# Data prepping

In [ ]:
tds = pd.read_csv('training_dataset.csv', sep=';')
ev_ds = pd.read_csv('evaluation_dataset.csv',sep=';')
f_ev_ds = pd.read_csv('final_evaluation_dataset.csv', sep=';')


In [ ]:
print('\n training dataset')
print(tds.head())
print('\n evaluation dataset------------')
print(ev_ds.head())
print('\n final evaluation dataset --------')
print(f_ev_ds.head())

In [ ]:
print('Training dataset')
print(tds.isna().sum())
print('\n eval dataset')
print(ev_ds.isna().sum())
print('\n final eval dataset')
print(f_ev_ds.isna().sum())

In [ ]:
#drop all NaN values.
f_ev_ds_noNAN = f_ev_ds.dropna()
tds_noNAN = tds.dropna()
ev_ds_noNAN = ev_ds.dropna()

In [ ]:
print(f'Org dataset amount:{len(tds)}')
print(f'Org dataset amount:{len(ev_ds)}')
print(f'Org dataset amount:{len(f_ev_ds)}')

print(f'Dropped Dataset amount:{len(tds_noNAN)}')
print(f'Dropped Dataset amount:{len(ev_ds_noNAN)}')
print(f'Dropped Dataset amount:{len(f_ev_ds_noNAN)}')

In [ ]:
#define what is congestion and what is not.
# congestion_flow = tds[tds['FLOW'] < 100]
# congestion_speed = tds[tds['SPEED_MS_AVG'] < 10]

ev_ds_noNAN['datetime'] = pd.to_datetime(ev_ds_noNAN['Date'].astype(str) + ' ' + ev_ds_noNAN['Time'])

ev_ds_offpeak = ev_ds_noNAN[(ev_ds_noNAN['datetime'].dt.time >= pd.to_datetime("04:00:00").time()) &
                            (ev_ds_noNAN['datetime'].dt.time <= pd.to_datetime("06:00:00").time())]
ev_ds_onpeak = ev_ds_noNAN[(ev_ds_noNAN['datetime'].dt.time >= pd.to_datetime("07:30:00").time()) &
                           (ev_ds_noNAN['datetime'].dt.time <= pd.to_datetime("08:30:00").time())]

ev_ds_offpeak = ev_ds_offpeak[(ev_ds_offpeak['FLOW'] > 0) &
                              (ev_ds_offpeak['SPEED_MS_AVG'].between(5, 50))]
ev_ds_onpeak = ev_ds_onpeak[(ev_ds_onpeak['FLOW'] > 0) &
                            (ev_ds_onpeak['SPEED_MS_AVG'].between(5, 50))]


In [ ]:
# converting the date and time columns int o just one "datetime" coluinm
tds_noNAN['datetime'] = pd.to_datetime(tds_noNAN['Date'].astype(str) + ' ' + tds_noNAN['Time'])

# filtering the dataset to only include the data within hours 04:00 and 06:00, this to get an accurate description of free flow speed.
tds_offpeak = tds_noNAN[(tds_noNAN['datetime'].dt.time >= pd.to_datetime("04:00:00").time()) &
                (tds_noNAN['datetime'].dt.time <= pd.to_datetime("06:00:00").time())]

tds_onpeak= tds_noNAN[(tds_noNAN['datetime'].dt.time >= pd.to_datetime("07:30:00").time()) &
                  (tds_noNAN['datetime'].dt.time <= pd.to_datetime("08:30:00").time())]
#Ensuring we only get correctly read speeds. E.g Removing any negative speeds and random slow drivers etc
#that do not actually depict the actual free flowspeed e.g people speeding and drinving super slow...
tds_offpeak = tds_offpeak[(tds_offpeak['FLOW'] > 0) &
                        (tds_offpeak['SPEED_MS_AVG'].between(5, 50))]
tds_onpeak = tds_onpeak[(tds_onpeak['FLOW'] > 0) &
                        (tds_onpeak['SPEED_MS_AVG'].between(5, 50))]
# Calculating the free flow speed
# only take the observations in the 85th quantile [m/s]
ffs_ms = (tds_offpeak['SPEED_MS_AVG'].quantile(0.85))
ffs_ms_2 = (tds_onpeak['SPEED_MS_AVG'].quantile(0.85))
# convert to km/h
ffs_kmh = (ffs_ms * 3.6)
ffs_kmh_2 =(ffs_ms_2 * 3.6)
print(f"Free Flow Speed [85th percentile]: \n Free flow speed: {ffs_ms_2 } [m/s] \n Free Flow Speed {ffs_kmh_2} [km/h]")
print(f"Free-flow speed (85th percentile): \n Free flow speed: {ffs_ms} [m/s] \n Free flow speed {ffs_kmh} [km/h]")

In [ ]:
# Calculate FFS (85th percentile)
ffs_ms_2 = tds_onpeak['SPEED_MS_AVG'].quantile(0.85)
ffs_kmh_2 = ffs_ms_2 * 3.6

plt.figure(figsize=(10,6))
plt.hist(tds_onpeak['SPEED_MS_AVG']*3.6, bins=40, color='skyblue', edgecolor='black', alpha=0.7)

plt.axvline(ffs_kmh_2, color='red', linestyle='--', linewidth=2, label=f'85th percentile (FFS = {ffs_kmh_2:.2f} km/h)')
plt.xlabel("Speed (km/h)")
plt.ylabel("Frequency")
plt.title("Distribution of Speeds During On-Peak (07:30–08:30)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Calculate FFS (85th percentile)
ffs_ms = tds_offpeak['SPEED_MS_AVG'].quantile(0.85)
ffs_kmh = ffs_ms * 3.6

# Plot histogram
plt.figure(figsize=(10,6))
plt.hist(tds_offpeak['SPEED_MS_AVG']*3.6, bins=40, color='skyblue', edgecolor='black', alpha=0.7)
plt.axvline(ffs_kmh, color='red', linestyle='--', linewidth=2, label=f'85th percentile (FFS = {ffs_kmh:.2f} km/h)')

plt.xlabel("Speed (km/h)")
plt.ylabel("Frequency")
plt.title("Distribution of Speeds During Off-Peak (04:00–06:00)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Define Congestion Threshold!
cong_th = ffs_kmh * 0.8 # why 70% of the ffs?
cong_th_2 = ffs_kmh_2 * 0.8

print(f'There is congestion at the avg. speed of: {cong_th} [km/h]')
print(f'There is congestion at the avg. speed of : {cong_th_2} [km/h]')

In [ ]:
# tds_noNAN['pred15'] = tds_noNAN['SPEED_MS_AVG'].shift(-1).rolling(15).mean()
# # congestion if 15 min av speed is less than cong_th (rn 60 km/h)
# tds_noNAN['congestion'] = (tds_noNAN['pred15']*3.6 <= cong_th).astype(int)

tds_noNAN['pred15'] = tds_noNAN['SPEED_MS_AVG'].shift(-1).rolling(15).mean()
tds_noNAN['congestion'] = (tds_noNAN['pred15']*3.6 <= cong_th_2).astype(int)

In [ ]:
def make_features_targets(df, horizon=15):


    df = df.copy()

    # --- Target: average speed in the next <horizon> minutes (leakage-safe) ---
    df[f'target_speed_{horizon}m'] = (
        df['SPEED_MS_AVG'].shift(-horizon).rolling(horizon).mean() * 3.6
    )

    # --- Feature engineering: past 15-min rolling stats ---
    df['past15_flow_mean']  = df['FLOW'].rolling(15).mean()
    df['past15_flow_std']   = df['FLOW'].rolling(15).std()
    df['past15_flow_min']   = df['FLOW'].rolling(15).min()
    df['past15_flow_max']   = df['FLOW'].rolling(15).max()
    df['past15_flow_slope'] = df['FLOW'] - df['FLOW'].shift(15)

    df['past15_speed_mean']  = df['SPEED_MS_AVG'].rolling(15).mean()
    df['past15_speed_std']   = df['SPEED_MS_AVG'].rolling(15).std()
    df['past15_speed_min']   = df['SPEED_MS_AVG'].rolling(15).min()
    df['past15_speed_max']   = df['SPEED_MS_AVG'].rolling(15).max()
    df['past15_speed_slope'] = df['SPEED_MS_AVG'] - df['SPEED_MS_AVG'].shift(15)

    # --- (Optional) Time-of-day cyclical features ---
    df["minute_of_day"] = df["datetime"].dt.hour * 60 + df["datetime"].dt.minute
    df["tod_sin"] = np.sin(2 * np.pi * df["minute_of_day"] / 1440)
    df["tod_cos"] = np.cos(2 * np.pi * df["minute_of_day"] / 1440)

    features = [
        'past15_flow_mean','past15_flow_std','past15_flow_min','past15_flow_max','past15_flow_slope',
        'past15_speed_mean','past15_speed_std','past15_speed_min','past15_speed_max','past15_speed_slope',
        'tod_sin','tod_cos'
    ]

    target = f'target_speed_{horizon}m'

    # Drop rows that can't compute full windows
    df = df.dropna(subset=features + [target])

    return df, features, target


In [ ]:
# 1. Create training data (for 2021)
tds_fixad, features, target = make_features_targets(tds_noNAN, horizon=15)
# 2. Create evaluation data (for 2022)
ev_fixad, _, _ = make_features_targets(ev_ds_noNAN, horizon=15)


In [ ]:
#Split into train / test sets
X = tds_fixad[features].values
y = tds_fixad[target].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
#Scaling the datat.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Models

In [ ]:
#perform linear regression model.
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

mae_lr = mean_absolute_error(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_lr = r2_score(y_test, y_pred_lr)
mape_lr = mean_absolute_percentage_error(y_test, y_pred_lr)

In [ ]:
# #Random forrest model
# random_f = RandomForestRegressor(n_estimators = 100, max_depth = 3, random_state = 42, n_jobs=-1)
# random_f.fit(X_train, y_train)
# y_pred_rf = random_f.predict(X_test)

In [ ]:
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
xgb_model.fit(X_train_scaled, y_train)

xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
mae_xgb= mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
mape_xgb = mean_absolute_percentage_error(y_test, y_pred_xgb)


In [ ]:
print("Linear Regression")
print(f"Mean Absolute Error (MAE): {mae_lr}")
print(f"Mean Squared Error (MSE): {mse_lr}")
print(f"Root Mean Squared Error (RMSE): {rmse_lr}")
print(f"R2-score {r2_lr}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_lr}")

print("\n")

print("XGBoost")
print(f"Mean Absolute Error (MAE): {mae_xgb}")
print(f"Mean Squared Error (MSE): {mse_xgb}")
print(f"Root Mean Squared Error (RMSE): {rmse_xgb}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_xgb}")
print(f"R2-score {r2_xgb}")
